In [ ]:
# Initial setup.
# Import numpy and pyplot. Set up notebook for interactive plotting.
# Also, increase the font size for figures, so we don't have to squint.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from matplotlib import rcParams
rcParams.update({'font.size': 14})


# Color vs temperature

The spectrum of stars can be approximated as a blackbody at the temperature of the stellar photosphere (about 5800 K for the Sun). Figure 2.1 from Maoz shows some examples of stellar spectra with effective temperatures ranging from 3500 to 40,000 K. While these spectra generally agree with the smooth blackbody curves, we can also see lots of additional structure from absorbtion in the outer layers of the stars.

<img src="http://wise-obs.tau.ac.il/~dani/astfigs/fig2.1.jpg" style="width: 600px;" />

Measuring detailed spectra like the ones shown above requires lots of telescope time with sensitive spectrographs, and can be impractical if you are trying to measure temperatures for a large number of stars. The Sloan Digital Sky Survey [Data Release 13](http://www.sdss.org/dr13/scope/) includes spectra for 851,968 stars but this uses data acquired over the course of 15 years on a dedicated 2.5m telescope!

A more efficient method to measure stellar temperatures is with multi-band photometry. A photometric filter only passes light in a certain range of wavelengths and has a well-defined transmission function. With a photometric filter in place, an imaging telescope can measure the apparent brightness of every object in its field of view. By imaging the same objects with multiple photometric filters, we can calculate their "colors" and derive effective temperatures.

## Photometric filter system

Standard photometric filter bands include U ("ultraviolet"), B ("blue"), V ("visible"), R ("red"), and I ("infrared"), though some telescopes use different systems. Files linked here contain normalized passbands for B, V, and R, taken from Table 2 of [Bessell,  PASP, 102, 1181 (1990)](http://adsabs.harvard.edu/abs/1990PASP..102.1181B).

* [bessell_B.dat](bessell_B.dat)
* [bessell_V.dat](bessell_V.dat)
* [bessell_R.dat](bessell_R.dat)

Read the filter functions from these files and make a plot of the filter responses vs wavelength. Recall that visible wavelengths range from &sim;390&ndash;700 nm.

In [ ]:
# Read filter functions (this part is done for you).
# Uses function np.genfromtxt to read numerical data from text file.
# Column 0 contains wavelength, column 1 contains response.
Bfilt = np.genfromtxt('bessell_B.dat')
Vfilt = np.genfromtxt('bessell_V.dat')
Rfilt = np.genfromtxt('bessell_R.dat')

# Plot filter response vs wavelength for inspection. 
# Use plt.plot function. Calling this function repeatedly will add additional lines to the same axes.
# Label your axes with plt.xlabel and plt.ylabel.


### Question 1: Describe the units / normalization used for the B, V, and R passbands from the Bessel reference.

*Write your answer here*


## Photometry

Next, write a function that takes a source spectrum (flux as a function of wavelength) and a filter function, and numerically integrates over wavelength to calculate the total flux measured in that photometric band. This integral represents the actual signal measured by the telescope&mdash;the source spectrum defines the flux from the source at each wavelength, then we multiply by the filter function to determine what fraction of that light passes into the camera, and we integrate to add up that light over all wavelengths. This function should be written so that it can apply to any type of spectrum, not just a blackbody. 

**Don't forget to pay attention to the units.** The filter functions that we found from the Bessell paper define wavelength in units of nm, but we will switch to units of cm to avoid confusion with the formulas from the textbook. The source spectrum is $f_\lambda$ with units of $\mathrm{erg} \, \mathrm{s}^{-1} \mathrm{cm}^{-2} \mathrm{cm}^{-1}$, which means "energy per second per square cm of telescope area per cm wavelength interval." After we integrate over wavelength, the returned value will be $f_{band}$ with units of $\mathrm{erg} \, \mathrm{s}^{-1} \mathrm{cm}^{-2}$. The calculation performed in this function doesn't actually know about the units used, but you will make your life a lot easier if you choose a specific convention for the units and stick to it.

In [ ]:
def photometric_flux(lmbda, filt, spectrum):
    """
    Calculate measured flux for a source observed with a photometric filter.
    
    Inputs
    ------
    lmbda : array, shape=(N,)
        Array of wavelength values, in units of cm.
    filt : array, shape=(N,)
        Filter response at each wavelength.
    spectrum : array
        Source spectrum at each wavelength, in units of erg s^{-1} cm^{-2} cm^{-1}.
    
    Returns
    -------
    flux : float
        Flux measured in a photometric band, in units of erg s^{-1} cm^{-2}.
    
    """
    
    # Multiply source spectrum by filter function and numerically integrate.
    # Try using the np.trapz function, which performs numerical integration using the trapezoidal rule.
    return flux


In [ ]:
# Test this function out by calculating the V-band flux from a source with a flat spectrum.
lmbda = Vfilt[:,0] * 1e-7 # Wavelength values for the filter function. Convert from nm to cm!
filt = Vfilt[:,1] # V-band filter function.
spectrum = np.ones(lmbda.shape) # Source spectrum is defined at each wavelength value, but always equal to one.
print(photometric_flux(lmbda, filt, spectrum))
# Value should be 8.9e-6 erg s^{-1} cm^{-2}.

# If we double the source spectrum, then photometric flux should double as well.
print(photometric_flux(lmbda, filt, 2 * spectrum))
# Value should be 1.8e-5 erg s^{-1} cm^{-2}.


## Color magnitude

To derive the color of an object, we can either take the ratio of the measured flux in two bands or the difference of the magnitudes&mdash;these are equivalent because magnitudes are logarithmic. Recall the formula for astronomical magnitudes:

$$
\begin{equation}
\mathrm{mag} - \mathrm{mag}_0 = -2.5 \log_{10} \left( \frac{\mathrm{flux}}{\mathrm{flux}_0} \right)
\end{equation}
$$

The expression for color is:

$$
\begin{equation}
\mathrm{color}_{A-B} = (\mathrm{mag}_A - \mathrm{mag}_{0,A}) - (\mathrm{mag}_B - \mathrm{mag}_{0,B})
\end{equation}
$$

The radius and distance of the object are the same for both measurements (labeled $A$ and $B$ above), so the distinction between absolute and apparent flux doesn't matter for color. However, it is important to keep track of the zero-points, which specify the reference flux for each photometric band. These can be affected by the normalization of the filter functions, which we inspected above.

Write a function to calculate color using two measured fluxes and their corresponding zero-points. We will leave the zero-point fluxes unspecified for now (they are inputs to your function) but will come back to them soon.

In [ ]:
def color(fluxA, fluxB, zpA, zpB):
    """
    Calculate color from measured flux in two bands.
    
    Inputs
    ------
    fluxA : float
        Measured flux in band A.
    fluxB : float
        Measured flux in band B.
    zpA : float
        Zero-point flux for band A.
    zpB : float
        Zero-point flux for band B.
        
    Returns
    -------
    clr : float
        Color magnitude calculated from the measurements provided.
    
    """
    
    # To calculate base-10 logarithms, use the function np.log10
    mA = -2.5 * np.log10(fluxA / zpA)
    mB = -2.5 * np.log10(fluxB / zpB)
    clr = mA - mB
    return clr


In [ ]:
# Test this function with a simple case.
# Assume the zero-point flux for bands A and B are both 1e10 erg s^{-1} cm^{-2}.
zpA = 1.0e10
zpB = 1.0e10
# Try the following values for the observed flux of an object in bands A and B.
fluxA = 0.9e9
fluxB = 0.5e10
# Calculate the color magnitude.
print(color(fluxA, fluxB, zpA, zpB))
# For this example, value should be 1.86


## Blackbody spectra

Our goal is to determine the relationship between color and temperature for blackbody spectra. Write a function to calculate the blackbody flux for a specified temperature. The blackbody flux calculated by this function will be used for the source spectrum input to your `photometric_flux` function, so you want the units to be $\mathrm{erg} \, \mathrm{s}^{-1} \mathrm{cm}^{-2} \mathrm{cm}^{-1}$, i.e. $f_\lambda (T)$.

In [ ]:
def blackbody(lmbda, T):
    """
    Calculate blackbody flux for temperature T at specified wavelengths.
    IMPORTANT: Make sure to use the expression for intensity/flux per *wavelength* interval, not frequency.
    
    Inputs
    ------
    lmbda : array
        Array of wavelength values to use, in units of cm.
    T : float
        Blackbody temperature, in units of Kelvin.
        
    Returns
    -------
    f : array
        Blackbody flux per unit wavelength, in units of erg s^{-1} cm^{-2} cm^{-1}. 
    
    """
    
    # Fundamental constants (find these near the front of your textbook).
    h = 6.6e-27 # erg s
    c = 3.0e10  # cm s^{-1}
    k = 1.4e-16 # erg K^{-1}
    # Add your code here to calculate blackbody flux.
    return f


In [ ]:
# To check your function, plot some blackbody spectra here for T = 3000K, 6000K, and 10000K.

# We will define a range of wavelengths with logarithmic spacing from 100 nm to 10000 nm (but convert to cm)
lmbda = np.logspace(2, 4, 100) # wavelengths ranging from 100 nm to 10000 nm
lmbda_cm = lmbda * 1e-7 # convert from nm to cm

# Use your blackbody function to calculate the spectra and plot them.
# Use plt.loglog to get a log-log plot, i.e. log(flux) vs log(wavelength)


## Magnitude reference

Now, let's return to the question of the zero-point fluxes. We need to know these before we can calculate the color magnitudes of observed objects. As we covered in lecture, the reference point for the astronomical magnitude system is the star Vega. So for any photometric band, the zero-point flux is equal to the flux of Vega.

Vega is a type A0V star (means that it is a blue-white main sequence star) with an effective temperature of 9600K. Use the `blackbody` and `photometric_flux` functions that you have written to calculate zero-point fluxes for $B$, $V$, and $R$ bands.

In [ ]:
# Calculate zero-point flux using Vega's effective temperature, 9600K.

# Check the values that you calculated.
# They will be large numbers (>1e10) in units of erg s^{-1} cm^{-2}.
print("zero-point(B) = {}".format(zpB))
print("zero-point(V) = {}".format(zpV))
print("zero-point(R) = {}".format(zpR))


### Question 2: What do the numerical values that you find for the B, V, and R band flux of Vega actually mean?

Obviously we do not actually receive power equal to $\sim 10^{10} \; \mathrm{erg} \, \mathrm{s}^{-1}$ on every square cm of our telescope. What factors were left out of this calculation? Explain why this oversight will not ultimately matter for our color calculation.

*Write your answer here*


## Color-temperature relation

Now that we have defined zero-point fluxes, we have all the tools needed to calculate $B-V$ color for blackbody sources of any temperature.

In [ ]:
# Calculate B-V color for blackbodies with temperature ranging from 3000K (type M stars) to 40000K (type O stars).
T = np.arange(3000.0, 40000.0, 1000.0)

# Makes an empty array (with the same size as the array of T value).
# Use this to store the calculated color magnitudes.
BVcolor = np.zeros(shape=T.shape)

# Loop over temperature values.
for i in range(len(T)):
    # Calculate flux in B band.

    # Calculate flux in V band.

    # Calculate B-V color (using zero-point fluxes that you calculated above).

# Plot B-V color vs temperature.
plt.plot(T, BVcolor)
plt.xlabel('T [K]')
plt.ylabel('$B-V$ color')


### Question 3: Based on the graph above, does $B-V$ color provide a more sensitive measurement of temperature at low (&lt;10000K) or high (&gt;20000K) temperatures? Explain how this difference in sensitivity relates to the shape of the blackbody spectrum.

*Write your answer here*

## Empirical relationship

While the code you developed above can be used to calculate the $B-V$ color for any temperature blackbody, it is useful to have a simpler formula that can be quickly applied. Also for observational data, color is the thing that we measure and temperature is the derived quantity, which means that the independent and dependent variables are opposite of the function we used above.

Try using the `np.polyfit` function with the results you have already calculated to derive a polynomial expression for temperature as a function of $B-V$ color. `np.polyfit` performs least squares regression to find the $n$th order polynomial that is the best fit to the data points provided. You can find documentation for how to use this function either by running `help(np.polyfit)` or by [looking it up on the web](http://lmgtfy.com/?q=numpy+polyfit).

Try fitting the color-temperature relation using polynomials of various orders and plotting the results. What difference do you find in the fits if you restrict the input data to a particular range of colors or temperatures? What is the domain of validity? (i.e. over what range of color does the polynomial agree with your data points to within X%?)

In [ ]:
# Use np.polyfit to find a polynomial expression for temperature as a function of B-V color.


In [ ]:
# Plot your polynomial function along with the color--temperature relationship that you calculated above.

# Also plot the fractional difference between the two, i.e. $(A - B) / A$.
# You can make a new set of plot axes using plt.figure


In a 2012 publication, Ballesteros provides the following equation that (approximately) relates $B-V$ color and temperature ([Ballesteros, Europhys. Lett., 97, 34008](http://adsabs.harvard.edu/abs/2012EL.....9734008B)).

$$
\begin{equation}
T = 4600K \times \left( \frac{1}{0.92 * (B-V) + 1.7} + \frac{1}{0.92 * (B-V) + 0.62} \right)
\end{equation}
$$


In [ ]:
# Make a similar plot to above, but show the fractional difference between:
#   a. the Ballesteros formula
#   b. your calculated color--temperature relationship
#   c. your polynomial functional form


### Question 4: What is the maximum fractional difference between the three curves? Which wavelength range shows the greatest disagreement?

*Write your answer here*

### Question 5: Can you think of some reasons why your calculated color--temperature relationship might be biased or inaccurate?

*Write your answer here*
